# Twitter API for sentiment Analysis (STREAMING API)

# Import the library

In [8]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import sys
import time


pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

# Authentication

In [13]:
consumer_key = "l6WroAAvOSe6469b8L51yMXHV" # Use your own key. To get a key https://apps.twitter.com/
consumer_secret = "c7qS6B4K3sNYA3p5MevjcKOYRt5cfuWf7gak4p74MuA2OGwdcA"
access_key = "15690986-UsKQeM4r7h4cZN3hRqfJfrxmIHfxSawvDZ26qEa11"
access_secret = "TXqq4EYcZv0LJuyVpWLNnDZBrFdnvDPD6pXFK0Ooe4KIh"

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tweepy.API(auth)

class CustomStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        #try:
            #Save live tweets to a csv file
            
            tweet = data.split(',"text":"')[1].split('","source')[0]
            saveThis = str(time.time())+'::'+tweet
            saveFile = open('TwitStream.csv','a')
            saveFile.write(saveThis)
            saveFile.write('\n')
            saveFile.close()
            return True
        #except BaseException, e:
         #   print 'failed ondata,'str(e)
            #time.sleep(5)
    
    #def on_status(self, status):
      #  print status.text

    def on_error(self, status_code):
        print >> sys.stderr, 'Encountered error with status code:', status_code
        return True # Don't kill the stream

    def on_timeout(self):
        print >> sys.stderr, 'Timeout...'
        return True # Don't kill the stream

tweetstream = tweepy.streaming.Stream(auth, CustomStreamListener())

# This is where you enter what you want to track
tweetstream.filter(track=['Chelsea'])



TypeError: Can't convert 'bytes' object to str implicitly

# STREAMING API

## Searching for Tweets

###`API.search(q[, lang][, locale][, rpp][, page][, since_id][, geocode][, show_user])`
```

    Returns tweets that match a specified query.
    Parameters:	

        q – the search query string
        lang – Restricts tweets to the given language, given by an ISO 639-1 code.
        locale – Specify the language of the query you are sending. This is intended for language-specific clients and the default should work in the majority of cases.
        rpp – The number of tweets to return per page, up to a max of 100.
        page – The page number (starting at 1) to return, up to a max of roughly 1500 results (based on rpp * page.
        geocode – Returns tweets by users located within a given radius of the given latitude/longitude. The location is preferentially taking from the Geotagging API, but will fall back to their Twitter profile. The parameter value is specified by “latitide,longitude,radius”, where radius units must be specified as either “mi” (miles) or “km” (kilometers). Note that you cannot use the near operator via the API to geocode arbitrary locations; however you can use this geocode parameter to search near geocodes directly.
        show_user – When true, prepends “<user>:” to the beginning of the tweet. This is useful for readers that do not display Atom’s author field. The default is false.

    Return type:	

    list of SearchResult objects
```

For data mining you will be dealing with a large amount of results. Cursor is a simple way to handle interation and results pages.

In [7]:
results = []
for tweet in tweepy.Cursor(api.search, q="Chelsea").items(100):
    results.append(tweet)

len(results)

100

### Store Results in a Data Frame

In [40]:
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_name"] = [tweet.author.name for tweet in results]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in results]
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set
data_set = process_results(results)

In [41]:
len(data_set)

1000

## Looking at the Data

In [42]:
data_set.head(5)

,id,text,created_at,retweet_count,favorite_count,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,user_location
0,655965902505013248,RT @FranCifelli: @Surfermom77 @HillaryClinton ...,2015-10-19 04:37:13,1,0,Twitter for iPhone,193357080,MyCrashIs,crazymama,2010-09-21 16:36:26,,253,1075,
1,655965901863391232,Goodmorning Chelsea Fans,2015-10-19 04:37:12,0,0,Twitter for Android,358160535,Tunnyking,Elvis Tunde,2011-08-19 13:43:06,CONGRATULATIONS... You're The 100th Person To ...,3086,1263,Fulham Road
2,655965900479201281,Pernikahan Mewah Chelsea Olivia Jadi Sorotan M...,2015-10-19 04:37:12,0,0,dlvr.it,259133172,dasomkiddo,lee dasom,2011-03-01 05:52:02,fake kiddo — pard of lee dasom ulzzang — 89lines,2576,2081,
3,655965896813383681,Pernikahan Mewah Chelsea Olivia Jadi Sorotan M...,2015-10-19 04:37:11,0,0,dlvr.it,993196357,Shine_SulliFx,Choi Jinri,2012-12-06 14:33:43,[VRP] of @ShineHouse_RP | fx's Sulli imnida♥ #...,224,18,
4,655965895391514624,dika kakina DikaKakina :\nPernikahan Mewah Che...,2015-10-19 04:37:11,0,0,IFTTT,3325436005,SenonGorosi,senon gorosi,2015-08-23 01:41:06,pepek senon,139,85,Republik Indonesien


In [43]:
data_set.to_csv('text2.csv')